In [1]:
from sqlalchemy import select
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column
from sqlalchemy.types import Text

In [2]:
class Base(DeclarativeBase):
    pass

class Message(Base):
    __tablename__ = "messages"

    id: Mapped[int] = mapped_column(primary_key=True)
    content: Mapped[str] = mapped_column(Text())

    def __repr__(self):
        return f"Message(id={self.id}, content={self.content})"

テーブル作れる。

In [3]:
url = "sqlite+aiosqlite:///example.db"

engine = create_async_engine(url=url, echo=True, echo_pool=True)
async with engine.begin() as conn:
    await conn.run_sync(Base.metadata.drop_all)
    await conn.run_sync(Base.metadata.create_all)

2024-03-14 21:03:47,264 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-14 21:03:47,264 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("messages")
2024-03-14 21:03:47,265 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-14 21:03:47,267 INFO sqlalchemy.engine.Engine 
DROP TABLE messages
2024-03-14 21:03:47,268 INFO sqlalchemy.engine.Engine [no key 0.00056s] ()
2024-03-14 21:03:47,271 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("messages")
2024-03-14 21:03:47,271 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-14 21:03:47,273 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("messages")
2024-03-14 21:03:47,274 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-14 21:03:47,276 INFO sqlalchemy.engine.Engine 
CREATE TABLE messages (
	id INTEGER NOT NULL, 
	content TEXT NOT NULL, 
	PRIMARY KEY (id)
)


2024-03-14 21:03:47,277 INFO sqlalchemy.engine.Engine [no key 0.00073s] ()
2024-03-14 21:03:47,279 INFO sqlalchemy.engine.Engine COMMIT


`add`して`commit`で`INSERT`。

In [4]:
session = AsyncSession(engine)
try:
    for content in ["foo", "bar", "baz", "qux"]:
        session.add(Message(content=content))
    await session.commit()
finally:
    await session.close()

2024-03-14 21:03:47,290 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-14 21:03:47,292 INFO sqlalchemy.engine.Engine INSERT INTO messages (content) VALUES (?) RETURNING id
2024-03-14 21:03:47,293 INFO sqlalchemy.engine.Engine [generated in 0.00012s (insertmanyvalues) 1/4 (ordered; batch not supported)] ('foo',)
2024-03-14 21:03:47,295 INFO sqlalchemy.engine.Engine INSERT INTO messages (content) VALUES (?) RETURNING id
2024-03-14 21:03:47,295 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/4 (ordered; batch not supported)] ('bar',)
2024-03-14 21:03:47,297 INFO sqlalchemy.engine.Engine INSERT INTO messages (content) VALUES (?) RETURNING id
2024-03-14 21:03:47,298 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/4 (ordered; batch not supported)] ('baz',)
2024-03-14 21:03:47,300 INFO sqlalchemy.engine.Engine INSERT INTO messages (content) VALUES (?) RETURNING id
2024-03-14 21:03:47,300 INFO sqlalchemy.engine.Engine [insertmanyvalues 4/4 (ordered; batch not supported)] ('qux',)
2

`get`で主キー指定して1件取得。

In [5]:
session = AsyncSession(engine)
try:
    m = await session.get(Message, 1)
    print(m)
finally:
    await session.close()

2024-03-14 21:03:47,314 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-14 21:03:47,316 INFO sqlalchemy.engine.Engine SELECT messages.id AS messages_id, messages.content AS messages_content 
FROM messages 
WHERE messages.id = ?
2024-03-14 21:03:47,317 INFO sqlalchemy.engine.Engine [generated in 0.00081s] (1,)
Message(id=1, content=foo)
2024-03-14 21:03:47,319 INFO sqlalchemy.engine.Engine ROLLBACK


個々の条件を`()`で括って`&`で`AND`。

In [6]:
session = AsyncSession(engine)
try:
    result = await session.execute(select(Message).where((1 < Message.id) & (Message.id < 4)))
    for m in result.scalars():
        print(m)
finally:
    await session.close()

2024-03-14 21:03:47,329 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-14 21:03:47,331 INFO sqlalchemy.engine.Engine SELECT messages.id, messages.content 
FROM messages 
WHERE messages.id > ? AND messages.id < ?
2024-03-14 21:03:47,332 INFO sqlalchemy.engine.Engine [generated in 0.00076s] (1, 4)
Message(id=2, content=bar)
Message(id=3, content=baz)
2024-03-14 21:03:47,335 INFO sqlalchemy.engine.Engine ROLLBACK


`|`で`OR`。

In [7]:
session = AsyncSession(engine)
try:
    result = await session.execute(select(Message).where((2 > Message.id) | (Message.id > 3)))
    for m in result.scalars():
        print(m)
finally:
    await session.close()

2024-03-14 21:03:47,345 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-14 21:03:47,346 INFO sqlalchemy.engine.Engine SELECT messages.id, messages.content 
FROM messages 
WHERE messages.id < ? OR messages.id > ?
2024-03-14 21:03:47,347 INFO sqlalchemy.engine.Engine [generated in 0.00078s] (2, 3)
Message(id=1, content=foo)
Message(id=4, content=qux)
2024-03-14 21:03:47,349 INFO sqlalchemy.engine.Engine ROLLBACK


In [8]:
await engine.dispose()

2024-03-14 21:03:47,357 INFO sqlalchemy.pool.impl.NullPool Pool recreating
